In [1]:
import sys
sys.path.append("../tools")
import drm

In [2]:
with open("../data/Deus Ex 3 - Director's Cut/files/bigfile.filelist") as f:
    names = f.read().split("\n")

with open("../data/Deus Ex 3 - Director's Cut/files/more.filelist") as f:
    names.extend(f.read().split("\n"))

assert names[0][0] == ";" # unimportant line
del names[0:1]

In [3]:
bigfile = drm.BigFile("../bigfiles/BIGFILE.000")

opened ../bigfiles/BIGFILE.000
opened ../bigfiles/BIGFILE.001
opened ../bigfiles/BIGFILE.002
opened ../bigfiles/BIGFILE.003
opened ../bigfiles/BIGFILE.004
opened ../bigfiles/BIGFILE.005
opened ../bigfiles/BIGFILE.006
opened ../bigfiles/BIGFILE.007
opened ../bigfiles/BIGFILE.008


In [4]:
print(len(names))
crchashes = set(bigfile.entries.keys())
knownhashes = {drm.crc32r(name.encode("utf-8")) for name in names}
print("{}/{}".format(len(crchashes & knownhashes), len(crchashes)))

33185
33182/33887


In [5]:
missing = []

for crc, payloads in bigfile.entries.items():
    if crc in knownhashes: continue

    # assume they all have the same type
    # so looking at the first is sufficient
    offset, size, lang = payloads[0]
    blob = bigfile.get_slice(offset, size)
    missing.append((crc, blob))

In [6]:
missing_fxanim = []
missing_fsb4 = []
missing_cdrm = []
missing_mus = []
missing_enic = []
missing_scaleform = []
missing_unknown = []

for crc, blob in missing:
    magic = blob[0:4]
    fxanim = b"FxAnim" in blob    # pc-w\audio\streams\vo\eng\ ... .mul
    fsb4 = b"FSB4" in blob        # pc-w\audio or music\ .mul .sam
    cdrm = magic == b"CDRM"       # everything
    mus = magic == b"Mus!"        # pc-w\music\ ... .mus
    enic = b"ENIC" in blob        # pc-w\art\cinematics\cinstream\ ... .mul
    scaleform = magic == b"CRID"  # pc-w/design_database/videos/cinematics/final_cinematics/cut_01_shq0_2_jaronnamir/dxni_110_v07-jaronnamir_assembly.usm

    #print("{:08x} {} {} {} {} {} {}".format(crc, fxanim, fsb4, cdrm, mus, enic, scaleform))
    
    if fxanim: missing_fxanim.append(crc)
    if fsb4: missing_fsb4.append(crc)
    if cdrm: missing_cdrm.append(crc)
    if mus: missing_mus.append(crc)
    if enic: missing_enic.append(crc)
    if scaleform: missing_scaleform.append(crc)
    if not (fxanim or fsb4 or cdrm or mus or enic or scaleform): missing_unknown.append(crc)

def print_group(header, entries):
    if len(entries) == 0: return
    print(header, len(entries))
    for entry in entries:
        print("{:08x}".format(entry))
    print()

# print_group("fxanim", missing_fxanim) # 89 entries
# print_group("fsb4", missing_fsb4) # 554 entries
# print_group("cdrm", missing_cdrm) # 143 entries
print_group("mus", missing_mus) # 0 entries
print_group("enic", missing_enic) # 1 entry
print_group("scaleform", missing_scaleform) # 1 entry
print_group("unknown", missing_unknown) # 8 entries

enic 1
24229121

scaleform 1
218fe0b6

unknown 8
0ff4b389
119ce2ca
341a6bb9
9c9d39cb
b9098414
b9dd84ae
c246e0cc
ea6cb8a4



# Unidentified file: 0ff4b389
This seems to descripe an animation related statemachine

In [7]:
print(dict(missing)[0x0ff4b389].decode("utf-8"))

1,91
1,uberobject_playanim
2,cover_transitiondone
3,cover_tochangefacing
4,movement_loopdone
5,cover_toswap
6,movement_transitiondone
7,weapon_transitiondone
8,weapon_reload_interrupt
9,weapon_fromwielding
10,weapon_toholster
11,weapon_todraw
12,weapon_aimtoreload
13,weapon_tofire
14,weapon_toreload
15,onskipline
16,onconversationgesturenonadditive
17,ongesturedone
18,default_hit
19,idleloop
20,onanimdone
21,onnpcsayline
22,onnpcintrodone
23,cancelhit
24,onconversationbodyanim
25,onintrodone
26,playeraction_done
27,knockdown
28,ongetup
29,onstopallactions
30,fullbody_hit_reaction
31,fullbody_hit_interrupt
32,genericaction_enter
33,genericaction_leave
34,onconversationexit
35,onconversationenter
36,simpleaction_enter
37,simpleaction_leave
38,simpleaction_interrupt
39,hit_emp_done
40,hitreactionmedium
41,onhitreactiondone
42,hitreaction
43,on_hit_emp
44,onreorientdone
45,onclaymoredone
46,ragdolldriveend
47,wakeupragdoll
48,death_enter
49,grenadepanic_leave
50,onstumblingdone
51,onrevive

# Unidentified file: 119ce2ca
An XML file describing the kind of build. The strings "patches" and "switches" do not appear in the game binary so it's unlikely this file is used by the game.

In [8]:
print(dict(missing)[0x119ce2ca].decode("utf-8"))

<dx3>
  <Version>2.0.779</Version>
  <Content>160928</Content>
  <patches>
    <Item Name="isPatch" Value="0"/>
    <Item Name="patchCode" Value=""/>
    <Item Name="patchContent" Value=""/>
  </patches>
  <switches>
    <Item Name="buildCode" Value="1"/>
    <Item Name="buildContent" Value="1"/>
    <Item Name="cleanBuildCode" Value="1"/>
    <Item Name="depFileParser" Value="1"/>
    <Item Name="deploySymbols" Value="1"/>
    <Item Name="buildLocalized" Value="1"/>
    <Item Name="analyzeResources" Value="1"/>
    <Item Name="createPs3Iso" Value="1"/>
    <Item Name="createBigFile" Value="1"/>
    <Item Name="deployQA" Value="1"/>
    <Item Name="deployDev" Value="1"/>
    <Item Name="triggerDemo" Value="1"/>
    <Item Name="applyCheckpoint" Value="1"/>
  </switches>
</dx3>



# Unidentified file: 341a6bb9
Unsure of the purpose, the names are of .drm files but not all of them actually exist in the DRM file.
`pc-w\base_alarm.drm` for example doesn't exist.

In [9]:
print(dict(missing)[0x341a6bb9].decode("utf-8")[:3000])

4,490
1,generalbank,firstobjects,generalbank
2,globalsoundinfo,firstobjects
3,globaldebuginfo,firstobjects,globaldebuginfo
4,globalaniminfo,firstobjects,globalaniminfo
5,globalscaleformdatabase,firstobjects
6,globaloutershell,firstobjects,globaloutershell
7,globaldlcoutershell,firstobjects,globaldlcoutershell
8,globaldlcinfo,firstobjects,globaldlcinfo
9,globaldatabase,firstobjects,globaldatabase
10,globaldlc,firstobjects,globaldlc
11,globalscripting,firstobjects
12,player_everyman,firstobjects,globalplayerinfo
13,globalplayerinfo,firstobjects,globalplayerinfo
14,particle,firstobjects
15,watervolume,firstobjects
16,globalloading,firstobjects,globalloading
17,globalsmartscripts,firstobjects
18,cinematic_adamjensen_commando,cinematic_adamjensen_commando,s_scn_cut_omg_thearrival_sin_omega_exterior
19,the_pod,s_scn_cut_omg_thearrival_sin_omega_exterior,the_pod
20,deferred_spot_rimlight,deferred_spot_rimlight,s_scn_global_light_conv_sin_omega_lab_biomec,s_scn_global_light_conv_sin_omega_lab_

In [10]:
import re
numeric = re.compile("\d+")
lines = dict(missing)[0x341a6bb9].decode("utf-8").replace("\r\n", "\n").split("\n")
pieces = {piece for line in lines for piece in line.split(",")}
pieces = {piece for piece in pieces if piece != "" and not numeric.match(piece)}
for piece in pieces:
    if piece[0] == "\\":
        path = "pc-w" + piece + ".drm"
    else:
        path = "pc-w\\" + piece + ".drm"
    crc = drm.crc32r(path.encode("utf-8"))
    if crc not in crchashes: # find those not in the BIGFILE
        print(path)

pc-w\base_automatic_animated_door.drm
pc-w\base_alarm_dispatcher.drm
pc-w\firstobjects.drm
pc-w\base_tv_screen.drm
pc-w\base_swinging_dbl_door.drm
pc-w\base_turret.drm
pc-w\base_security_door.drm
pc-w\base_searchable.drm
pc-w\base_computer_shopping.drm
pc-w\base_faridah_chopper.drm
pc-w\base_computer.drm
pc-w\base_searchable_animated.drm
pc-w\base_breakable_window.drm
pc-w\base_external_disc.drm
pc-w\base_laserbeam.drm
pc-w\base_playanim_ontrigger.drm
pc-w\base_alarm.drm
pc-w\base_alarm_panel.drm
pc-w\base_player_interact_door.drm
pc-w\base_news_reader.drm
pc-w\base_keypad.drm
pc-w\base_punchtroughwall_spawndebris.drm
pc-w\base_ireader.drm
pc-w\base_dedicated_terminal.drm
pc-w\base_generic_switch.drm
pc-w\base_bee_chopper.drm
pc-w\base_swinging_door.drm
pc-w\base_dstr_fireextinguisher.drm
pc-w\base_call_elevator.drm
pc-w\base_auto_horizontal_sliding_dbl_door.drm


# Unidentified file: 9c9d39cb
Various filenames

In [11]:
print(dict(missing)[0x9c9d39cb].decode("utf-8")[:10000])

1,17535
1,admd://d:\dx3dlc-pc\area\scenarios\scn_cut_omg_thearrival\overlays\s_scn_cut_omg_thearrival_sin_omega_exterior
2,d:\dx3dlc-pc\area\scenarios\scn_cut_omg_thearrival\overlays\s_scn_cut_omg_thearrival_sin_omega_exterior.dat
3,area/scenarios/scn_cut_omg_thearrival/overlays/s_scn_cut_omg_thearrival_sin_omega_exterior/s_scn_cut_omg_thearrival_sin_omega_exterior
4,d:\dx3dlc-pc\art\fsfx\basiclvl\underwater.fsfx
5,game/script/core/core/clock
6,game/script/core/core/corebase
7,game/script/core/core/messagechannel
8,game/script/core/core/rotation
9,game/script/core/core/symbol
10,game/script/core/core/vector
11,game/script/game/game/action
12,game/script/game/game/actiontype
13,game/script/game/game/actor
14,game/script/game/game/actorsensor
15,game/script/game/game/actortunedata
16,game/script/game/game/actortype
17,game/script/game/game/aimingcontroller
18,game/script/game/game/anchor
19,game/script/game/game/animation
20,game/script/game/game/animgraphrtvalue
21,game/script/game/game

# Unidentified file: b9098414
Filenames for anim fragments maybe

In [12]:
print(dict(missing)[0xb9098414].decode("utf-8")[:10000])

6,5899
1,d:\dx3dlc-pc\object\projectiles\thrownproj\thrownproj_remote_detonated\ani\idle_free_30_1_0_0_0_0_-1
2,d:\dx3dlc-pc\object\pickups\equip\base\weapon_common\ani\idle_free_30_1_0_0_0_0_-1
3,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpm_idle_free_30_1_0_0_0_0_0
4,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_1_1_0_2
5,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_2_2_0_-1
6,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_2_2_0_2
7,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_3_3_0_2
8,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_4_4_0_5
9,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_5_5_0_-1
10,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_5_5_0_5
11,d:\dx3dlc-pc\animation\game\p\cbt\doublebarrelshotgun\wpn_aim_free_30_1_0_6_6_0_5
12,d:\dx3dlc-pc\animation\game\p\cbt\doubleb

# Unidentified file: b9dd84ae
I'd have thought everyone uses utf-8 these days, but there is one line here that goes `video\scaleform\sharedtextures\credits\35_février2009` where the é is encoded as hex 0xE9, which is how ISO 8859-1 does it.

This looks very much like a list of texture filenames before they got converted into the final formats.
Might be interesting to see if there is an unexplained id value close to the material or texture files that lines up with this list.

In [13]:
# print(dict(missing)[0xb9dd84ae].decode("utf-8"))
print(dict(missing)[0xb9dd84ae].decode("latin1")[:10000])

6,2667
1,art\texture_library\cube_map\det_city_sarif_cube
2,art\textures\default_mat_txt\flatmask
3,art\textures\default_mat_txt\flatnormal
4,art\textures\default_mat_txt\flatwhite
5,cdc\assets\textures\dummyblack
6,object\characters\base\mesh\_kit_systems_\eyes\adam_eye|d
7,object\characters\base\mesh\_kit_systems_\eyes\eyes_n|n
8,object\characters\base\mesh\_shared_parts\eye\textures\eye_lens_d
9,object\characters\base\mesh\_shared_parts\eye\textures\eye_new|specdots
10,object\characters\base\mesh\_shared_parts\facial_hairs\facial_hairs_males|beards_d
11,object\characters\base\mesh\adam_jensen\art\textures\adam_jensen_arms_d|d
12,object\characters\base\mesh\adam_jensen\art\textures\adam_jensen_arms_m|m
13,object\characters\base\mesh\adam_jensen\art\textures\adam_jensen_arms_n|n
14,object\characters\base\mesh\adam_jensen\art\textures\adam_jensen_arms_s|s
15,object\characters\base\mesh\adam_jensen\art\textures\adam_jensen_fp_aug_d
16,object\characters\base\mesh\adam_jensen\art\textures

# Unidentified file: c246e0cc
Joint names

In [14]:
print(dict(missing)[0xc246e0cc].decode("utf-8"))

1,454
1,t
2,r
3,waistjoint
4,r_hipjoint
5,r_kneejoint
6,r_anklejoint
7,r_balljoint
8,r_hiprolljoint
9,l_hipjoint
10,l_kneejoint
11,l_anklejoint
12,l_balljoint
13,l_hiprolljoint
14,spine0joint
15,spine1joint
16,spine2joint
17,neck0joint
18,neckrolljoint
19,headjoint
20,jawjoint
21,tongue1joint
22,tongue0joint
23,c_lowerlipjoint
24,r_lowerlipjoint
25,r_lowerlip2joint
26,l_lowerlipjoint
27,l_lowerlip2joint
28,c_chinjoint
29,c_btmchinjoint
30,r_lowercheekjoint
31,r_midjawjoint
32,r_chinjoint
33,l_lowercheekjoint
34,l_midjawjoint
35,l_chinjoint
36,r_lowermouthjoint
37,l_lowermouthjoint
38,r_outjawjoint
39,l_outjawjoint
40,c_uppermouthjoint
41,c_upperlipjoint
42,l_upperlidjoint
43,l_eyejoint
44,r_eyejoint
45,r_upperlidjoint
46,c_browjoint
47,c_forheadjoint
48,c_nosetopjoint
49,c_nosetipjoint
50,r_forheadtopjoint
51,r_outtermouthjoint
52,r_outtercheekjoint
53,r_nosesneerjoint
54,r_topcheekjoint
55,r_incheekjoint
56,r_nosewrinclejoint
57,r_eyelidtopjoint
58,r_eyeoutjoint
59,r_eyelid2joint
60,r

# Unidentified file: ea6cb8a4
Sound names

In [15]:
print(dict(missing)[0xea6cb8a4].decode("utf-8")[:10000])

3,1851
1,foley\object\physic\barrel_plastic_slide_01
2,foley\object\physic\remotedetonategel\remotedetonategel_01
3,foley\object\physic\remotedetonategel\remotedetonategel_02
4,foley\object\physic\remotedetonategel\remotedetonategel_03
5,foley\pickup\genericpickup
6,foley\object\squib\grenades\feedback_01
7,foley\object\squib\grenades\feedback_02
8,foley\object\squib\grenades\feedback_03
9,foley\object\squib\grenades\grenade_frag_august_02
10,foley\object\squib\grenades\reverse_whoosh
11,foley\pickup\ammo\sniper\pickup_ammo_sniper_01
12,foley\pickup\ammo\sniper\pickup_ammo_sniper_02
13,foley\pickup\ammo\sniper\pickup_ammo_sniper_03
14,foley\object\physic\grenade_frag\frag_grenade_bounce_01
15,foley\object\physic\grenade_frag\frag_grenade_bounce_02
16,foley\object\physic\grenade_frag\frag_grenade_roll
17,foley\object\squib\aud\aud_beep_01
18,foley\object\squib\aud\aud_buildup_01
19,foley\object\squib\aud\aud_explosion_01
20,foley\object\physic\weapon\gundrop_bounce
21,foley\pickup\weapo